In [1]:
from IPython.core.display import HTML 

# Introduction to MongoDB with PyMongo and NOAA Data

This notebook provides a basic walkthrough of how to use MongoDB and is based on a tutorial originally by [Alberto Negron](http://altons.github.io/python/2013/01/21/gentle-introduction-to-mongodb-using-pymongo/).

Metadata records are frequently stored as JSON, and almost anything you get from an API will be JSON. For example, check out the [metadata records](https://data.noaa.gov/data.json) for the National Oceanic and Atmospheric Administration. 

MongoDB is a great tool to use with JSON data because it stores structured data as JSON-like documents, using dynamic schemas (called BSON), rather than predefined schemas. 

In MongoDB, an element of data is called a document, and documents are stored in collections. One collection may have any number of documents. Collections are a bit like tables in a relational database, and documents are like records. But there is one big difference: every record in a table has the same fields (with, usually, differing values) in the same order, while each document in a collection can have completely different fields from the other documents.

Documents are Python dictionaries that can have strings as keys and can contain various primitive types (int, float,unicode, datetime) as well as other documents (Python dicts) and arrays (Python lists).

## Getting started
First we need to import `json` and `pymongo`.

Note that the `pprint` module provides a capability to “pretty-print” arbitrary Python data structures in a form which can be used as input to the interpreter. This is particularly helpful with JSON. You can read more about `pprint` [here](https://docs.python.org/2/library/pprint.html).

In [2]:
import json
import pymongo
from pprint import pprint

## Connect    
Just as with the relational database example with `sqlite`, we need to begin by setting up a connection. With MongoDB, we will be using `pymongo`, though MongoDB also comes with a [console API that uses Javascript](https://docs.mongodb.org/manual/tutorial/write-scripts-for-the-mongo-shell/).    


To make our connection, we will use the PyMongo method `MongoClient`:

In [3]:
conn=pymongo.MongoClient()
conn

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

## Create and access a database    
Mongodb creates databases and collections automatically for you if they don't exist already. A single instance of MongoDB can support multiple independent databases. When working with PyMongo, we access databases using attribute style access, just like we did with `sqlite`:

In [4]:
db = conn.mydb
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'mydb')

## Collections    
A collection is a group of documents stored in MongoDB, and can be thought of as roughly the equivalent of a table in a relational database. Getting a collection in PyMongo works the same as getting a database:

In [5]:
collection = db.my_collection
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'mydb'), u'my_collection')

## Insert data   
To insert some data into MongoDB, all we need to do is create a dict and call `insert_one` on the collection object:

In [6]:
doc = {"class":"XBUS-502","date":"03-05-2016","instructor":"Bengfort","classroom":"C222","roster_count":"25"}
collection.insert_one(doc)

In [7]:
conn.database_names()

[u'local', u'mydb']

In [8]:
db.collection_names()

[u'my_collection', u'system.indexes']

## A practical example

Let's say you wanted to gather up a bunch of JSON metadata records and store them for analysis. 

```python
import requests

NOAA_URL = "https://data.noaa.gov/data.json"

def load_data(URL):
    """
    Loads the data from URL and returns data in JSON format.
    """
    r = requests.get(URL)
    data = r.json()
    return data
    
noaa = load_data(NOAA_URL)
```

This takes a long time, so I've created a file for you that contains a small chunk of the records to use for today's workshop.

In [ ]:
with open("data_sample.json") as data_file:    
    noaa = json.load(data_file)

In [ ]:
len(noaa)

### Checking out the data
Now let's print out just one record to examine the structure. 

In [ ]:
pprint(noaa[0]['description'])

### Define the database
We will want to enter these records into our database. But first, we'll define a specific database for the NOAA records:

In [ ]:
db = conn.earthwindfire

### Define the collection
Next we define the collection where we'll insert the NOAA metadata records:

In [ ]:
records = db.records

### Insert data    
Then we loop through each record in the NOAA dataset and insert just the target information for each into the collection:

In [ ]:
def insert(metadata):
    for dataset in metadata:
        data ={}
        data["title"] = dataset["title"]
        data["description"] = dataset["description"]
        data["keywords"] = dataset["keyword"]
        data["accessLevel"] = dataset["accessLevel"]
        # choose your own
        # choose your own
        # choose your own 
        # choose your own

        records.insert_one(data)

insert(noaa)

In [ ]:
records.count()

### Querying 

#### Querying with `.findOne( )`    
The find_one() method selects and returns a single document from a collection and returns that document (or None if there are no matches). It is useful when you know there is only one matching document, or are only interested in the first match

In [ ]:
records.find_one()

#### Querying with `.find( )`
To get more than a single document as the result of a query we use the `find()` method. `find()` returns a Cursor instance, which allows us to iterate over all matching documents.

```python
records.find()
```

For example, we can iterate over the first 2 documents (there are a lot in the collection and this is just an example) in the records collection

In [ ]:
for rec in records.find()[:2]:
    pprint(rec)

#### Searching 
MongoDB queries are represented as JSON-like structure, just like documents. To build a query, you just need to specify a dictionary with the properties you wish the results to match. For example, let's say we were just interested in publically available satellite data from [NESDIS](http://www.nesdis.noaa.gov/).

This query will match all documents in the records collection with keywords code "NESDIS".

In [ ]:
records.find({"keywords": "NESDIS"}).count()

We can further narrow our search by adding additional fields

In [ ]:
records.find({"keywords": "NESDIS","keywords": "Russia","accessLevel":"public"}).count()

### Statistics

In [ ]:
db.command({'dbstats': 1})

In [ ]:
db.command({'collstats': 'records'})

### The aggregation pipeline 

The [aggregation pipeline](https://docs.mongodb.org/manual/core/aggregation-pipeline/) allows MongoDB to provide native aggregation capabilities that corresponds to many common data aggregation operations in SQL.

The following table provides an overview of common SQL aggregation terms, functions, and concepts and the corresponding MongoDB aggregation operators:    
    
| SQL Terms, Functions, and Concepts  | MongoDB Aggregation Operators  |
| ----------------------------------  |:-------------------------------|
| WHERE                               | \$match                        |
| GROUP BY                            | \$group                        |
| HAVING                              | \$match                        |
| SELECT	                          | \$project                      |
| ORDER BY	                          | \$sort                         |
| LIMIT                               | \$limit                        |
| SUM()   	                          | \$sum                          |
| COUNT()	                          | \$sum                          |
| join	                              | \$lookup                       |

In [ ]:
from IPython.display import Image
Image(filename='images/mongodb_pipeline.png', width=600, height=300)

### Removing data

In [ ]:
conn.earthwindfire.collection_names()

In [ ]:
# Delete our records collection
conn.earthwindfire.drop_collection("records")
conn.earthwindfire.collection_names()

In [ ]:
conn.database_names()

In [ ]:
# Remove the earthwindfire database
conn.drop_database("earthwindfire")
conn.database_names()